In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from IPython.display import display

# Load the data from the provided Excel file
data_path = "data.xlsx"  # Ganti dengan path file kamu
data_df = pd.read_excel(data_path)

# Display the loaded data
display(data_df.head(10))

# Encode 'Jenis Kelamin'  [Laki laki  = 0, Perenpuan = 1]
label_encoder = LabelEncoder()
data_df['Jenis Kelamin'] = label_encoder.fit_transform(data_df['Jenis Kelamin'])
# Filter the relevant columns for features and labels
X = data_df[['Jenis Kelamin', 'Berat Badan Saat Lahir (kg)', 'Tinggi Badan Saat Lahir (cm)', 
             'Berat Badan Saat Ini (kg)', 'Tinggi Badan Saat Ini (cm)', 'Usia (bulan)', 
             'Z-Score Berat Badan', 'Z-Score Tinggi Badan']].values
y = data_df['Status Gizi'].apply(lambda x: 1 if x == 'Gizi buruk (severely wasted)' else -1).values  # Binary labels

# Display the features and labels
print("Features (X):")
print(X[:10])
print("\nLabels (y):")
print(y[:10])


In [ ]:
import numpy as np

# Initialize parameters
w = np.zeros(X.shape[1])  # Weight vector
b = 0  # Bias term
learning_rate = 0.001  # Learning rate
n_iters = 1000  # Number of iterations

# Display the initialized parameters
print("Initial weight vector (w):", w)
print("Initial bias (b):", b)
print("Learning rate:", learning_rate)
print("Number of iterations:", n_iters)


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Pastikan X dan y adalah numpy array untuk operasi aritmatika yang benar
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32).reshape(-1, 1)

# Scaling fitur-fitur data
scaler = StandardScaler()
X = scaler.fit_transform(X).astype(np.float32)

# Inisialisasi parameter
n_samples, n_features = X.shape
learning_rate = 0.01
n_iters = 1000

# Membuat variabel untuk bobot dan bias
w = tf.Variable(tf.zeros([n_features, 1], dtype=tf.float32))
b = tf.Variable(0.0, dtype=tf.float32)

# Fungsi untuk menghitung margin
def compute_margin(X, y, w, b):
    return y * (tf.matmul(X, w) + b)

# Fungsi untuk menghitung loss SVM
def svm_loss(X, y, w, b, C=1):
    margins = compute_margin(X, y, w, b)
    hinge_loss = tf.reduce_mean(tf.maximum(0.0, 1 - margins))
    regularization_loss = tf.reduce_sum(tf.square(w))
    return hinge_loss + C * regularization_loss

# Optimizer
optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

# Proses pelatihan menggunakan TensorFlow
for epoch in range(1, n_iters + 1):
    with tf.GradientTape() as tape:
        loss = svm_loss(X, y, w, b)
    
    gradients = tape.gradient(loss, [w, b])
    optimizer.apply_gradients(zip(gradients, [w, b]))
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: loss = {loss.numpy()}, w = {w.numpy().flatten()}, b = {b.numpy()}")

# Tampilkan parameter model akhir
print(f"Parameter model akhir: w = {w.numpy().flatten()}, b = {b.numpy()}")

# Fungsi prediksi
def predict(X, w, b):
    return tf.sign(tf.matmul(X, w) + b)

# Prediksi pada set pelatihan
predictions = predict(X, w, b)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predictions, y), tf.float32))

print(f"Akurasi pelatihan: {accuracy.numpy() * 100:.2f}%")


In [ ]:
import pickle

# Simpan model ke dalam file menggunakan pickle
model_path = "svm_model.pkl"
with open(model_path, 'wb') as model_file:
    pickle.dump((w, b), model_file)

# Informasikan bahwa model telah disimpan
print(f"Model telah disimpan ke {model_path}")


In [ ]:
# Membuat 20 data dummy untuk pengujian
data_dummy = pd.DataFrame({
    'Jenis Kelamin': [0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1],  # 0 untuk 'L', 1 untuk 'P'
    'Berat Badan Saat Lahir (kg)': [3.5, 2.8, 3.2, 3.0, 3.1, 3.4, 3.3, 2.9, 3.6, 3.1, 3.0, 2.7, 3.4, 3.3, 3.2, 3.5, 3.0, 3.6, 2.9, 3.4],
    'Tinggi Badan Saat Lahir (cm)': [50, 48, 52, 49, 50, 51, 53, 47, 52, 49, 48, 46, 51, 50, 48, 49, 50, 52, 47, 51],
    'Berat Badan Saat Ini (kg)': [7.0, 6.2, 8.0, 7.5, 7.3, 7.8, 7.4, 6.5, 8.2, 7.1, 6.9, 6.0, 7.6, 7.3, 7.0, 7.5, 7.2, 8.0, 6.4, 7.8],
    'Tinggi Badan Saat Ini (cm)': [60, 58, 62, 61, 60, 63, 61, 59, 64, 61, 60, 57, 62, 60, 59, 62, 60, 63, 58, 61],
    'Usia (bulan)': [12, 10, 14, 13, 11, 13, 12, 9, 15, 12, 11, 8, 14, 11, 10, 12, 11, 15, 9, 13],
    'Z-Score Berat Badan': [0.5, -0.5, 1.0, 0.8, 0.7, 0.9, 0.6, -0.3, 1.2, 0.4, 0.2, -0.7, 0.9, 0.6, 0.5, 0.8, 0.3, 1.1, -0.4, 0.7],
    'Z-Score Tinggi Badan': [0.4, -0.3, 0.9, 0.7, 0.5, 0.8, 0.6, -0.4, 1.0, 0.3, 0.1, -0.6, 0.8, 0.5, 0.4, 0.7, 0.2, 0.9, -0.5, 0.6]
})

# Display the dummy data
display(data_dummy)

In [ ]:
# Fungsi prediksi
def predict(X, w, b):
    return np.dot(X, w) + b

# Membaca model yang disimpan
model_path = "svm_model.pkl"
with open(model_path, 'rb') as model_file:
    w, b = pickle.load(model_file)

# Menampilkan parameter model yang dibaca
print(f"Parameter model yang dibaca: w = {w}, b = {b}")

# Memilih fitur yang relevan dari data dummy untuk prediksi
X_dummy = data_dummy[['Jenis Kelamin', 'Berat Badan Saat Lahir (kg)', 
                      'Tinggi Badan Saat Lahir (cm)', 'Berat Badan Saat Ini (kg)', 
                      'Tinggi Badan Saat Ini (cm)', 'Usia (bulan)', 
                      'Z-Score Berat Badan', 'Z-Score Tinggi Badan']].values

# Melakukan prediksi pada data dummy
predictions = predict(X_dummy, w, b)
binary_predictions = np.sign(predictions)  # Konversi prediksi menjadi -1 atau 1

# Print detail prediksi untuk penjelasan
print("Detail Prediksi:")
for i, (x, pred) in enumerate(zip(X_dummy, predictions)):
    print(f"Data {i+1}: {x}")
    print(f"Nilai SVM: {pred}")
    print(f"Status Gizi: {'Gizi buruk (severely wasted)' if binary_predictions[i] == 1 else 'Bukan gizi buruk'}")
    print()

# Menentukan status gizi berdasarkan prediksi
status_gizi_predictions = ['Gizi buruk' if pred == 1 else 'Bukan gizi buruk' for pred in binary_predictions]

# Menambahkan kolom hasil prediksi dan status gizi ke data dummy
data_dummy['Nilai SVM'] = predictions
data_dummy['Prediksi Status Gizi'] = status_gizi_predictions

# Menampilkan hasil prediksi
display(data_dummy)
